In [7]:
from imutils import contours
from skimage import measure
import numpy as np
import imutils
import cv2
import pandas as pd
import glob
import swifter

import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
def get_folder(path):
    return path.split('/')[-2]

def get_file(path):
    return path.split('/')[-1]

In [9]:
df['folder'] = df.path.swifter.progress_bar(True).apply(get_folder)

In [10]:
df['file'] = df.path.swifter.progress_bar(True).apply(get_file)

In [11]:
df

,path,raises_exception,folder,file
0,../dataset_generation/kaggle_dataset/0/189.jpg,True,0,189.jpg
1,../dataset_generation/kaggle_dataset/0/162.jpg,True,0,162.jpg
2,../dataset_generation/kaggle_dataset/0/176.jpg,True,0,176.jpg
3,../dataset_generation/kaggle_dataset/0/610.jpg,True,0,610.jpg
4,../dataset_generation/kaggle_dataset/0/360.jpg,True,0,360.jpg
...,...,...,...,...
645,../dataset_generation/kaggle_dataset/1/580.jpg,True,1,580.jpg
646,../dataset_generation/kaggle_dataset/1/345.jpg,True,1,345.jpg
647,../dataset_generation/kaggle_dataset/1/386.jpg,True,1,386.jpg
648,../dataset_generation/kaggle_dataset/1/609.jpg,True,1,609.jpg


In [12]:
def get_disk(row):
  try:
    image = row.path
    folder = row.folder
    file = row.file
    image = cv2.imread(image)
    
    width = 512
    dim = (width, width)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    crop = 100
    image = image[100:width-100, 100:width-100]
    
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.bilateralFilter(gray,9,75,75)
    median=cv2.medianBlur(blur, 5)

    # threshold the image to reveal light regions in the
    # blurred image
    thresh = cv2.threshold(median, 155, 255, cv2.THRESH_BINARY)[1]
    # perform a series of erosions and dilations to remove
    # any small blobs of noise from the thresholded image
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=4)

    # perform a connected component analysis on the thresholded
    # image, then initialize a mask to store only the "large"
    # components
    labels = measure.label(thresh, connectivity=2, background=0)
    mask = np.zeros(thresh.shape, dtype="uint8")

    # loop over the unique components
    for label in np.unique(labels):
      # if this is the background label, ignore it
      if label == 0:
        continue

      # otherwise, construct the label mask and count the
      # number of pixels 
      labelMask = np.zeros(thresh.shape, dtype="uint8")
      labelMask[labels == label] = 255
      numPixels = cv2.countNonZero(labelMask)

      # if the number of pixels in the component is sufficiently
      # large, then add it to our mask of "large blobs"
      if numPixels >300:
        mask = cv2.add(mask, labelMask)

    # find the contours in the mask, then sort them from left to
    # right
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
      cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = contours.sort_contours(cnts)[0]
    # print (cnts)
    # loop over the contours
    for (i, c) in enumerate(cnts):
        ellipse = cv2.fitEllipse(c)
        (x, y, w, h) = cv2.boundingRect(c)
        break
      # draw the bright spot on the image
    padding = 0
    image = image[y-padding:y+h+padding, x-padding:x+w+padding]
    color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(color_coverted)
    pil_image.save("disk_dataset_kaggle_2/"+str(folder)+"/"+file)
  except:
    print (row.file)

In [13]:
df.head()

,path,raises_exception,folder,file
0,../dataset_generation/kaggle_dataset/0/189.jpg,True,0,189.jpg
1,../dataset_generation/kaggle_dataset/0/162.jpg,True,0,162.jpg
2,../dataset_generation/kaggle_dataset/0/176.jpg,True,0,176.jpg
3,../dataset_generation/kaggle_dataset/0/610.jpg,True,0,610.jpg
4,../dataset_generation/kaggle_dataset/0/360.jpg,True,0,360.jpg


In [14]:
df.swifter.progress_bar(True).apply(get_disk, axis=1)

(pid=13160) 233.jpg


0      None
1      None
2      None
3      None
4      None
       ... 
645    None
646    None
647    None
648    None
649    None
Length: 650, dtype: object